In [ ]:
import os

import pandas as pd

from data import Dataset, split_transfer_dataset, get_random_datasets
from learner import PredictorLearner, TransferLearner
from analysis import get_wilcoxon_p_value

# define training set sizes to be used in experiments
TRAINING_SET_SIZES = [0.2, 0.4, 0.6, 0.8]
SUBJECT_SYSTEMS = ['NODEJS', 'X264', 'XZ']
REPETITIONS = 5

print('Initialising results tables')
# initialise results columns
rq1_results_fields = ['mse_accuracy_tgt_no_cv', 'mape_accuracy_tgt_no_cv', 'mse_accuracy_tgt_cv', 'mape_accuracy_tgt_cv',
                     'mse_accuracy_trans_no_cv', 'mape_accuracy_trans_no_cv', 'mse_accuracy_trans_cv', 'mape_accuracy_trans_cv']
rq1_results = pd.DataFrame(columns=rq1_results_fields)
rq2_results_fields = ['mape_accuracy_pred_20pct', 'mape_accuracy_pred_40pct', 'mape_accuracy_pred_60pct', 
                     'mape_accuracy_pred_80pct', 'mape_accuracy_trans_20pct', 'mape_accuracy_trans_40pct', 
                     'mape_accuracy_trans_60pct',  'mape_accuracy_trans_80pct', 'mse_accuracy_pred_20pct', 
                     'mse_accuracy_pred_40pct', 'mse_accuracy_pred_60pct', 'mse_accuracy_pred_80pct',  
                     'mse_accuracy_trans_20pct', 'mse_accuracy_trans_40pct', 'mse_accuracy_trans_60pct', 
                     'mse_accuracy_trans_80pct']
rq2_results = pd.DataFrame(columns=rq2_results_fields)
rq3_results_fields = ['training_time_pred_20pct_no_cv', 'training_time_pred_40pct_no_cv', 'training_time_pred_60pct_no_cv', 
                     'training_time_pred_80pct_no_cv', 'training_time_pred_20pct_cv', 'training_time_pred_40pct_cv', 
                     'training_time_pred_60pct_cv', 'training_time_pred_80pct_cv', 'training_time_trans_20pct_no_cv', 
                     'training_time_trans_40pct_no_cv', 'training_time_trans_60pct_no_cv', 'training_time_trans_80pct_no_cv', 
                     'training_time_trans_20pct_cv', 'training_time_trans_40pct_cv', 'training_time_trans_60pct_cv', 
                     'training_time_trans_80pct_cv']
rq3_results = pd.DataFrame(columns=rq3_results_fields)
print('Results tabled initialised\n')

print('Selecting random datasets')
# randomly select a source and target dataset for each subject system
datasets = get_random_datasets()
print('Datasets selected')

In [ ]:
### RQ1

predictor = PredictorLearner()
transferrer = TransferLearner()

for subject_system in SUBJECT_SYSTEMS:
    print('***********************************\nSubject system:', subject_system)
    
    # delete the results of previous subject system's test before entering next subject system's results
    mape_accuracy_pred_no_cv = []
    mape_accuracy_trans_no_cv = []
    mape_accuracy_pred_cv = []
    mape_accuracy_trans_cv = []
    mse_accuracy_pred_no_cv = []
    mse_accuracy_trans_no_cv = []
    mse_accuracy_pred_cv = []
    mse_accuracy_trans_cv = []

    # get the randomly selected target and source datasets for the current subject system
    subject_system_datasets = datasets[subject_system]

    for rep in range(REPETITIONS):
        print('Experiment repetition', rep+1)
        # grab a target and source dataset for this experiment repetition
        src_dataset, tgt_dataset = subject_system_datasets[rep]
        print('Source dataset path:', src_dataset.get_csv_path())
        print('Target dataset path:', tgt_dataset.get_csv_path(),'\n')

        # get optimised predictor model using hyperparameter optimisation
        X_train, X_validate, y_train, y_validate = tgt_dataset.get_split_dataset()
        optimised_model = predictor.get_optimal_params(X_validate, y_validate)

        # get accuracy of optimised model for predictor learner
        predictor.fit(X_train, y_train, premade_model=optimised_model)
        mape_accuracy_pred_cv.append(predictor.get_error(X_train, y_train, measure='mape'))
        mse_accuracy_pred_cv.append(predictor.get_error(X_train, y_train, measure='mse'))

        # get accuracy of non-optimised model for predictor learner
        predictor.fit(X_train, y_train)
        mape_accuracy_pred_no_cv.append(predictor.get_error(X_train, y_train, measure='mape'))
        mse_accuracy_pred_no_cv.append(predictor.get_error(X_train, y_train, measure='mse'))


        # get optimised transfer model using hyperparameter optimisation
        X_train, X_validate, y_train, y_validate = split_transfer_dataset(src_dataset, tgt_dataset)
        optimised_model = transferrer.get_optimal_params(X_validate, y_validate)

        # get accuracy of optimised model for transfer learner
        transferrer.fit(X_train, y_train, premade_model=optimised_model)
        mape_accuracy_trans_cv.append(transferrer.get_error(X_train, y_train, measure='mape'))
        mse_accuracy_trans_cv.append(predictor.get_error(X_train, y_train, measure='mse'))

        # get accuracy of non-optimised model for transfer learner
        transferrer.fit(X_train, y_train)
        mape_accuracy_trans_no_cv.append(transferrer.get_error(X_train, y_train, measure='mape'))
        mse_accuracy_trans_no_cv.append(predictor.get_error(X_train, y_train, measure='mse'))


    rq1_results['mse_accuracy_tgt_no_cv'] = mse_accuracy_pred_no_cv
    rq1_results['mape_accuracy_tgt_no_cv'] = mape_accuracy_pred_no_cv
    rq1_results['mse_accuracy_tgt_cv'] = mse_accuracy_pred_cv
    rq1_results['mape_accuracy_tgt_cv'] = mape_accuracy_pred_cv
    rq1_results['mse_accuracy_trans_no_cv'] = mse_accuracy_trans_no_cv
    rq1_results['mape_accuracy_trans_no_cv'] = mape_accuracy_trans_no_cv
    rq1_results['mse_accuracy_trans_cv'] = mse_accuracy_trans_cv
    rq1_results['mape_accuracy_trans_cv'] = mape_accuracy_trans_cv
    
    print('Wilcoxon p value:', get_wilcoxon_p_value(mape_accuracy_pred_cv, mape_accuracy_trans_cv), '\n')

    # create results path if it isn't present and save results to csv
    output_dir = os.path.join('../results/', subject_system.lower())
    print('Writing output to', os.path.join(output_dir, 'rq1.csv'), '\n')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    rq1_results.to_csv(os.path.join(output_dir, 'rq1.csv'), index=False)
    print('Output written to', os.path.join(output_dir, 'rq1.csv'), '\n')

    # reset dataframe after results are saved to csv
    rq1_results = pd.DataFrame(columns=rq1_results.columns)

In [ ]:
### RQ2

transferrer = TransferLearner()
predictor = PredictorLearner()

for subject_system in SUBJECT_SYSTEMS:
    print('***********************************\nSubject system:', subject_system)    
    # delete the results of previous subject system's test before entering next subject system's results
    mape_accuracy_pred_20pct = []
    mape_accuracy_pred_40pct = []
    mape_accuracy_pred_60pct = []
    mape_accuracy_pred_80pct = []
    mape_accuracy_trans_20pct = []
    mape_accuracy_trans_40pct = []
    mape_accuracy_trans_60pct = []
    mape_accuracy_trans_80pct = []
    mse_accuracy_pred_20pct = []
    mse_accuracy_pred_40pct = []
    mse_accuracy_pred_60pct = []
    mse_accuracy_pred_80pct = []
    mse_accuracy_trans_20pct = []
    mse_accuracy_trans_40pct = []
    mse_accuracy_trans_60pct = []
    mse_accuracy_trans_80pct = []

    # get the randomly selected target and source datasets for the current subject system
    subject_system_datasets = datasets[subject_system]

    for rep in range(REPETITIONS):
        print('Experiment repetition', rep+1)
        # grab a target and source dataset for this experiment repetition
        src_dataset, tgt_dataset = subject_system_datasets[rep]
        print('Source dataset path:', src_dataset.get_csv_path())
        print('Target dataset path:', tgt_dataset.get_csv_path(),'\n')

        for train_size in TRAINING_SET_SIZES:

            X_train, X_validate, y_train, y_validate = tgt_dataset.get_split_dataset()
            optimised_model = predictor.get_optimal_params(X_validate, y_validate)

            # get accuracy of predictor model for current training set size
            predictor.fit(X_train, y_train, premade_model=optimised_model)
            mape_accuracy_pred = predictor.get_error(X_train, y_train, measure='mape')
            mse_accuracy_pred = predictor.get_error(X_train, y_train, measure='mse')


            X_train, X_validate, y_train, y_validate = split_transfer_dataset(src_dataset, 
                                                                              tgt_dataset, 
                                                                              train_size=train_size)
            optimised_model = transferrer.get_optimal_params(X_validate, y_validate)

            # get accuracy of transfer model for each training set size
            transferrer.fit(X_train, y_train, premade_model=optimised_model)
            mape_accuracy_trans = transferrer.get_error(X_train, y_train, measure='mape')
            mse_accuracy_trans = transferrer.get_error(X_train, y_train, measure='mse')


            # record accuracy in appropriate results column
            if train_size == 0.2:
                mape_accuracy_pred_20pct.append(mape_accuracy_pred)
                mape_accuracy_trans_20pct.append(mape_accuracy_trans)
                mse_accuracy_pred_20pct.append(mse_accuracy_pred)
                mse_accuracy_trans_20pct.append(mse_accuracy_trans)
            elif train_size == 0.4:
                mape_accuracy_pred_40pct.append(mape_accuracy_pred)
                mape_accuracy_trans_40pct.append(mape_accuracy_trans)
                mse_accuracy_pred_40pct.append(mse_accuracy_pred)
                mse_accuracy_trans_40pct.append(mse_accuracy_trans)
            elif train_size == 0.6:
                mape_accuracy_pred_60pct.append(mape_accuracy_pred)
                mape_accuracy_trans_60pct.append(mape_accuracy_trans)
                mse_accuracy_pred_60pct.append(mse_accuracy_pred)
                mse_accuracy_trans_60pct.append(mse_accuracy_trans)
            elif train_size == 0.8:
                mape_accuracy_pred_80pct.append(mape_accuracy_pred)
                mape_accuracy_trans_80pct.append(mape_accuracy_trans)
                mse_accuracy_pred_80pct.append(mse_accuracy_pred)
                mse_accuracy_trans_80pct.append(mse_accuracy_trans)


    rq2_results['mape_accuracy_pred_20pct'] = mape_accuracy_pred_20pct
    rq2_results['mape_accuracy_pred_40pct'] = mape_accuracy_pred_40pct
    rq2_results['mape_accuracy_pred_60pct'] = mape_accuracy_pred_60pct
    rq2_results['mape_accuracy_pred_80pct'] = mape_accuracy_pred_80pct
    rq2_results['mse_accuracy_pred_20pct'] = mse_accuracy_pred_20pct
    rq2_results['mse_accuracy_pred_40pct'] = mse_accuracy_pred_40pct
    rq2_results['mse_accuracy_pred_60pct'] = mse_accuracy_pred_60pct
    rq2_results['mse_accuracy_pred_80pct'] = mse_accuracy_pred_80pct
    rq2_results['mape_accuracy_trans_20pct'] = mape_accuracy_trans_20pct
    rq2_results['mape_accuracy_trans_40pct'] = mape_accuracy_trans_40pct
    rq2_results['mape_accuracy_trans_60pct'] = mape_accuracy_trans_60pct
    rq2_results['mape_accuracy_trans_80pct'] = mape_accuracy_trans_80pct
    rq2_results['mse_accuracy_trans_20pct'] = mse_accuracy_trans_20pct
    rq2_results['mse_accuracy_trans_40pct'] = mse_accuracy_trans_40pct
    rq2_results['mse_accuracy_trans_60pct'] = mse_accuracy_trans_60pct
    rq2_results['mse_accuracy_trans_80pct'] = mse_accuracy_trans_80pct
    
    print('Wilcoxon p value:', get_wilcoxon_p_value(mape_accuracy_trans_80pct, mape_accuracy_trans_20pct), '\n')
    
    # create results path if it isn't present and save results to csv
    output_dir = os.path.join('../results/', subject_system.lower())
    print('Writing output to', os.path.join(output_dir, 'rq2.csv'), '\n')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    rq2_results.to_csv(os.path.join(output_dir, 'rq2.csv'), index=False)
    print('Output written to', os.path.join(output_dir, 'rq2.csv'), '\n')
    
    # reset dataframe after results are saved to csv
    rq2_results = pd.DataFrame(columns=rq2_results.columns)

In [ ]:
### RQ3

transferrer = TransferLearner()
predictor = PredictorLearner()

for subject_system in SUBJECT_SYSTEMS:
    print('***********************************\nSubject system:', subject_system)
    # delete the results of previous subject system's test before entering next subject system's results
    training_time_pred_20pct_no_cv = []
    training_time_pred_40pct_no_cv = []
    training_time_pred_60pct_no_cv = []
    training_time_pred_80pct_no_cv = []
    training_time_pred_20pct_cv = []
    training_time_pred_40pct_cv = []
    training_time_pred_60pct_cv = []
    training_time_pred_80pct_cv = []
    training_time_trans_20pct_no_cv = []
    training_time_trans_40pct_no_cv = []
    training_time_trans_60pct_no_cv = []
    training_time_trans_80pct_no_cv = []
    training_time_trans_20pct_cv = []
    training_time_trans_40pct_cv = []
    training_time_trans_60pct_cv = []
    training_time_trans_80pct_cv = []

    # get the randomly selected target and source datasets for the current subject system
    subject_system_datasets = datasets[subject_system]

    for rep in range(REPETITIONS):
        print('Experiment repetition', rep+1)
        # grab a target and source dataset for this experiment repetition
        src_dataset, tgt_dataset = subject_system_datasets[rep]
        print('Source dataset path:', src_dataset.get_csv_path())
        print('Target dataset path:', tgt_dataset.get_csv_path(),'\n')

        for train_size in TRAINING_SET_SIZES:

            X_train, X_validate, y_train, y_validate = tgt_dataset.get_split_dataset()
            # get optimised predictor model using hyperparameter optimisation
            optimised_model = predictor.get_optimal_params(X_validate, y_validate)
            predictor.fit(X_train, y_train, premade_model=optimised_model)

            # gather results
            training_time_pred_no_cv = predictor.get_training_time()
            training_time_pred_cv = predictor.get_training_time(include_optimisation_time=True)


            X_train, X_validate, y_train, y_validate = split_transfer_dataset(src_dataset, tgt_dataset)
            # get optimised transfer model using hyperparameter optimisation
            optimised_model = transferrer.get_optimal_params(X_validate, y_validate)
            transferrer.fit(X_train, y_train, premade_model=optimised_model)

            # gather results
            training_time_trans_no_cv = transferrer.get_training_time()
            training_time_trans_cv = transferrer.get_training_time(include_optimisation_time=True)


            # record accuracy in appropriate results column
            if train_size == 0.2:
                training_time_pred_20pct_no_cv.append(training_time_pred_no_cv)
                training_time_pred_20pct_cv.append(training_time_pred_cv)
                training_time_trans_20pct_no_cv.append(training_time_trans_no_cv)
                training_time_trans_20pct_cv.append(training_time_trans_cv)
            elif train_size == 0.4:
                training_time_pred_40pct_no_cv.append(training_time_pred_no_cv)
                training_time_pred_40pct_cv.append(training_time_pred_cv)
                training_time_trans_40pct_no_cv.append(training_time_trans_no_cv)
                training_time_trans_40pct_cv.append(training_time_trans_cv)
            elif train_size == 0.6:
                training_time_pred_60pct_no_cv.append(training_time_pred_no_cv)
                training_time_pred_60pct_cv.append(training_time_pred_cv)
                training_time_trans_60pct_no_cv.append(training_time_trans_no_cv)
                training_time_trans_60pct_cv.append(training_time_trans_cv)
            elif train_size == 0.8:
                training_time_pred_80pct_no_cv.append(training_time_pred_no_cv)
                training_time_pred_80pct_cv.append(training_time_pred_cv)
                training_time_trans_80pct_no_cv.append(training_time_trans_no_cv)
                training_time_trans_80pct_cv.append(training_time_trans_cv)

    print('Wilcoxon p value for no CV:', get_wilcoxon_p_value(training_time_pred_80pct_no_cv, training_time_trans_80pct_no_cv))
    print('Wilcoxon p value for CV:', get_wilcoxon_p_value(training_time_pred_80pct_cv, training_time_trans_80pct_cv), '\n')

    rq3_results['training_time_pred_20pct_no_cv'] = training_time_pred_20pct_no_cv
    rq3_results['training_time_pred_40pct_no_cv'] = training_time_pred_40pct_no_cv
    rq3_results['training_time_pred_60pct_no_cv'] = training_time_pred_60pct_no_cv
    rq3_results['training_time_pred_80pct_no_cv'] = training_time_pred_80pct_no_cv
    rq3_results['training_time_pred_20pct_cv'] = training_time_pred_20pct_cv
    rq3_results['training_time_pred_40pct_cv'] = training_time_pred_40pct_cv
    rq3_results['training_time_pred_60pct_cv'] = training_time_pred_60pct_cv
    rq3_results['training_time_pred_80pct_cv'] = training_time_pred_80pct_cv
    rq3_results['training_time_trans_20pct_no_cv'] = training_time_trans_20pct_no_cv
    rq3_results['training_time_trans_40pct_no_cv'] = training_time_trans_40pct_no_cv
    rq3_results['training_time_trans_60pct_no_cv'] = training_time_trans_60pct_no_cv
    rq3_results['training_time_trans_80pct_no_cv'] = training_time_trans_80pct_no_cv
    rq3_results['training_time_trans_20pct_cv'] = training_time_trans_20pct_cv
    rq3_results['training_time_trans_40pct_cv'] = training_time_trans_40pct_cv
    rq3_results['training_time_trans_60pct_cv'] = training_time_trans_60pct_cv
    rq3_results['training_time_trans_80pct_cv'] = training_time_trans_80pct_cv

    # create results path if it isn't present and save results to csv
    output_dir = os.path.join('../results/', subject_system.lower())
    print('Writing output to', os.path.join(output_dir, 'rq3.csv'), '\n')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    rq3_results.to_csv(os.path.join(output_dir, 'rq3.csv'), index=False)
    print('Output written to', os.path.join(output_dir, 'rq3.csv'), '\n')

    # reset dataframe after results are saved to csv
    rq3_results = pd.DataFrame(columns=rq3_results.columns)